# Data Profiling
# 0. Set up

In [102]:
# import libraries
import pandas as pd
import seaborn as sns
from cryptography.fernet import Fernet

# load data
df = pd.read_csv('input/dirty-loan-data.csv')
df.shape

(50100, 60)

In [103]:
df.head(5)

,Unnamed: 0,id_pk,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,next_pymnt_year,last_credit_pull_month,last_credit_pull_year,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
0,0,55978502,59640239,14400.0,14400.0,14400.0,36,5.61,50000.0,D,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,15278.0,15000.0
1,1,56141977,59803813,24550.0,24550.0,24550.0,36,2.29,50000.0,C,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,163368.0,41400.0
2,2,18935279,21167968,20000.0,20000.0,20000.0,60,6.29,50000.0,D,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,35408.0,28300.0
3,3,27580978,30084088,15000.0,15000.0,15000.0,36,-3.97,50000.0,A,...,NaN,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,146372.0,81600.0
4,4,4536779,5779043,16000.0,16000.0,15950.0,36,-1.10,50000.0,A,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,355820.0,45600.0


# 1. Create Data Profile

In [104]:
from ydata_profiling import ProfileReport

profile = ProfileReport(df, title='Loan Dataset Profiling Report', minimal=True)
profile.to_file(output_file='output/profile/loan_data_profile.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

# 1.1. Analyze Integrity
## 1.1.1. Duplicates
To identify duplicates the first step is to remove the first column, which simply contains a row index. If we do not remove this column, the profiling will identify no rows as duplicates.

In [105]:
# drop first column
df = df.drop(columns='Unnamed: 0')

Looking at the output generated by the profiling, the first obvious problem is that the ID variables (**id_pk**, **id_member**) do not seem to be unique. This might be due to duplicated entries in the dataset. We can check this by filtering for all entries containing duplicated 'member_id' and order by 'member_id'.

In [106]:
# filter for all entries containing duplicated 'member_id' order by member_id
df[df.duplicated(subset='member_id', keep=False)].sort_values(by='member_id')

,id_pk,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,next_pymnt_year,last_credit_pull_month,last_credit_pull_year,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
50023,435307,520078,7000.0,7000.0,7000.000000,36,4.96,242.52,D,D2,...,NaN,Jul,2015.0,0.0,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN
23,435307,520078,7000.0,7000.0,7000.000000,36,4.96,242.52,D,D2,...,NaN,Jul,2015.0,0.0,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN
50011,526784,681457,11200.0,11200.0,11164.159885,36,3.23,378.62,C,C1,...,NaN,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN
11,526784,681457,11200.0,11200.0,11164.159885,36,3.23,378.62,C,C1,...,NaN,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN
50,558201,718601,7200.0,7200.0,7150.000000,60,7.56,181.12,E,E4,...,NaN,Feb,2015.0,0.0,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53,67325141,72126853,28800.0,28800.0,28800.000000,60,0.64,621.03,B,B4,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,292439.0,36100.0
61,67367721,72179492,3000.0,3000.0,3000.000000,36,7.57,107.82,D,D4,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,101900.0,15300.0
50061,67367721,72179492,3000.0,3000.0,3000.000000,36,7.57,107.82,D,D4,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,101900.0,15300.0
50034,67828497,72687218,5000.0,5000.0,5000.000000,36,-4.68,150.58,A,A1,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,45434.0,59700.0


Additionally we can also check for rows where all variables are identical. This will return a boolean series where True indicates that the row is a duplicate of a previous row. We can then sum the number of True values to get the number of duplicates.

In [107]:
# show number of duplicates where all rows are equal
df.duplicated().sum()

95

Looking at the output we can see that we indeed have 95 duplicated cases (here the duplicates are identified by rows that contain identical values for all variables). However, we have 200 rows with duplicated member IDs (Here we identify duplicates based on members IDs only). This means that we have some cases where the member ID is duplicated, but the other variables are not identical. This is a problem, as it indicates that we have some inconsistencies in the data. Now let's start by removing the rows where all variables are identical.

In [108]:
# remove duplicates
df = df.drop_duplicates()
df.shape

(50005, 59)

Now we can re-check if we still have duplicates in the member_id variable. As we can see in the output below this is indeed the case. The mistake seems to lie in the variable **installment** where we have differing values for the same case. This is likely a data error, as the variable **installment** should contain the monthly payment owed by the borrower. This means that the value 50000 is likely a mistake, as it is unlikely that a borrower would have to pay 50000 per month. We should therefore remove duplicated cases where installment have a very high value.

In [109]:
# filter for all entries containing duplicated 'member_id' order by member_id and installment
df[df.duplicated(subset='member_id', keep=False)].sort_values(by=['member_id', 'installment'])

,id_pk,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,next_pymnt_year,last_credit_pull_month,last_credit_pull_year,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
50004,4536779,5779043,16000.0,16000.0,15950.0,36,-1.10,508.06,A,A5,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,355820.0,45600.0
4,4536779,5779043,16000.0,16000.0,15950.0,36,-1.10,50000.00,A,A5,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,355820.0,45600.0
50002,18935279,21167968,20000.0,20000.0,20000.0,60,6.29,489.45,D,D2,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,35408.0,28300.0
2,18935279,21167968,20000.0,20000.0,20000.0,60,6.29,50000.00,D,D2,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,35408.0,28300.0
50003,27580978,30084088,15000.0,15000.0,15000.0,36,-3.97,456.54,A,A1,...,NaN,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,146372.0,81600.0
3,27580978,30084088,15000.0,15000.0,15000.0,36,-3.97,50000.00,A,A1,...,NaN,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,146372.0,81600.0
50000,55978502,59640239,14400.0,14400.0,14400.0,36,5.61,503.50,D,D1,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,15278.0,15000.0
0,55978502,59640239,14400.0,14400.0,14400.0,36,5.61,50000.00,D,D1,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,15278.0,15000.0
50001,56141977,59803813,24550.0,24550.0,24550.0,36,2.29,818.82,C,C1,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,163368.0,41400.0
1,56141977,59803813,24550.0,24550.0,24550.0,36,2.29,50000.00,C,C1,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,163368.0,41400.0


In [110]:
# drop row contains duplicate member_id and installment differs, remove row with highest installment
df = df.sort_values(by='installment', ascending=True).drop_duplicates(subset='member_id', keep='first')
# check if all duplicates in member_id are removed
df[df.duplicated(subset='member_id', keep=False)].shape

(0, 59)

In [111]:
# show row for member id 5779043
df[df['member_id'] == 5779043]

,id_pk,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,next_pymnt_year,last_credit_pull_month,last_credit_pull_year,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
50004,4536779,5779043,16000.0,16000.0,15950.0,36,-1.1,508.06,A,A5,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,355820.0,45600.0


## 1.1.2. Inconsistencies
Another problem that becomes apparent when looking at the data is the discrepancy between the variables **grade** and **subgrade**. The **subgrade** variable is a subset of the **grade** variable. We can check this by looking at the unique values for both variables. However, we can see that the **subgrade** variable contains the nonsensical value *ALPHA_CENTAURI*. We need to address this issue by, first removing this value.

In [112]:
# show unique values for grade and subgrade
(df['grade'].unique(), df['sub_grade'].unique())

(array(['D', 'A', 'B', 'C', 'G', 'E', 'F'], dtype=object),
 array(['D2', 'A2', 'A3', 'A1', 'A4', 'B1', 'B2', 'ALPHA_CENTAURI', 'B5',
        'B4', 'A5', 'C1', 'C2', 'C3', 'C4', 'C5', 'D1', 'D4', 'D3', 'G1',
        'D5', 'E2', 'E1', 'E3', 'F5', 'E4', 'E5', 'F1', 'F4', 'F2', 'F3',
        'G2', 'G3', 'G4', 'G5'], dtype=object))

In [113]:
# replace ALPHA_CENTAURI from sub_grade with None
df['sub_grade'] = df['sub_grade'].replace('ALPHA_CENTAURI', None)

# show unique values for grade and subgrade
df['sub_grade'].unique()

array(['D2', 'A2', 'A3', 'A1', 'A4', 'B1', 'B2', None, 'B5', 'B4', 'A5',
       'C1', 'C2', 'C3', 'C4', 'C5', 'D1', 'D4', 'D3', 'G1', 'D5', 'E2',
       'E1', 'E3', 'F5', 'E4', 'E5', 'F1', 'F4', 'F2', 'F3', 'G2', 'G3',
       'G4', 'G5'], dtype=object)

In [114]:
# show rows for which sub_grade is NULL
df[df['sub_grade'].isnull()]

,id_pk,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,next_pymnt_year,last_credit_pull_month,last_credit_pull_year,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
20588,66631610,71357375,1000.0,1000.0,1000.0,36,-0.24,32.16,B,None,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,7305.0,17600.0
22176,65976695,70671480,1000.0,1000.0,1000.0,36,-0.24,32.16,B,None,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,9376.0,17400.0
21684,68443647,73333393,1000.0,1000.0,1000.0,36,-0.20,32.18,B,None,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,146562.0,17400.0
13212,61722545,65892267,1000.0,1000.0,1000.0,36,-0.01,32.27,B,None,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,13254.0,11700.0
48383,59230236,63117999,1000.0,1000.0,1000.0,36,-0.01,32.27,B,None,...,NaN,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,785.0,7600.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25357,10105780,11957868,35000.0,35000.0,34900.0,36,1.99,1162.34,B,None,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,121149.0,77600.0
24824,8606497,10358554,35000.0,35000.0,34950.0,36,1.99,1162.34,B,None,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,25854.0,73000.0
40753,12386926,14399043,35000.0,35000.0,35000.0,36,1.99,1162.34,B,None,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,217660.0,73500.0
24446,14699830,16762170,35000.0,35000.0,35000.0,36,1.99,1162.34,B,None,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,210478.0,49100.0


Should we apply a cluster analysis to impute values for subgrade?

# 1.2. Analyze Completeness
## 1.2.1. Missing Values

# 1.3. Analyze Accuracy
## 1.3.1. Data Types and Formats
Another problem identified in the data are that at the moment dates are split into years and month. This becomes first visible when looking at the variables **issue_year** and **issue_month** as an example. Since we only have observations for 2015, this means that the **issue_year** variable by itself does not provide any useful information (no variation) in future analytical applications. Only in combination with the month, does the year provide useful information.

In [115]:
# show distribution of issue_year
sns.countplot(data=df, x='issue_year')

<Axes: xlabel='issue_year', ylabel='count'>

Contrary to **issue_year** the variable **issue_month** does show some variation, as shown in the plot below.

In [116]:
# show distribution of issue_month
sns.countplot(data=df, x='issue_month')

<Axes: xlabel='issue_month', ylabel='count'>

However, we also see that *October* was misspelled as *Octxyz*. We can fix this by replacing the misspelled value with the correct one.

In [117]:
# replace misspelled value
df['issue_month'] = df['issue_month'].replace('Octxyz', 'Oct')
df['issue_month'].unique()

array(['Nov', 'Oct', 'Jan', 'Aug', 'Feb', 'Jun', 'Dec', 'Apr', 'Mar',
       'Sep', 'May', 'Jul'], dtype=object)

To remedy the fact that issue_year contains no variation, but we still would like to know the year, we can create a new variable that combines the year and month into a single variable. This will allow us to keep the information about the year, but also the month. The same transformation can be applied to the other time variables in the dataset (earliest_cr_line_month, earliest_cr_line_year, last_pymnt_month, last_pymnt_year, next_pymnt_month, next_pymnt_year, last_credit_pull_month, last_credit_pull_year).

In [118]:
# create new variable issue_date with data type datetime
df['issue_date'] = pd.to_datetime(df['issue_year'].astype(str) + '-' + df['issue_month'], format='%Y-%b')

# drop issue_year and issue_month
df = df.drop(columns=['issue_year', 'issue_month'])

# show first 5 rows
df['issue_date'].head(5)

34866   2015-11-01
2735    2015-10-01
40836   2015-01-01
21990   2015-08-01
4759    2015-10-01
Name: issue_date, dtype: datetime64[ns]

# 1.4. Text Issues

# 1.5. Data Protection
We might want to encrypt some variables that contain sensitive information, such as the url. This is because the url might contain information that could be used to identify the borrower. In the case that we need to be able to convert the data back to its original form, we should use encryption. With encryption are able to use a key to encrypt and decrypt data.

In [130]:
# generate private key
private_key = Fernet.generate_key()

# create fernet object
fernet = Fernet(private_key)

# Ensure that 'url' column is of type string if not convert it
if isinstance(df['url'], str) is False:
    df['url'] = df['url'].astype(str)   

# encrypt url
df['url'] = df['url'].apply(lambda x: fernet.encrypt(x.encode()))
df[['member_id', 'url']].head(5)

,member_id,url
34866,787218,b'gAAAAABl2JfYqbCNidH11d7eogUAHC22ikRApZSudjP9...
2735,764324,b'gAAAAABl2JfYQGx6wR-Giyi-t5-CK3bXajbUpcl9KYaq...
40836,835461,b'gAAAAABl2JfY4fIjhAuw1EWK4FY6SS97Sefp84vg8Y5D...
21990,1080466,b'gAAAAABl2JfYihZa65mM06TlZBArDGIM3bcsTpPbgwbH...
4759,67800109,b'gAAAAABl2JfYtslzJranJacxGTsDubRvW3NIkAjlRLvN...


In [131]:
# decrypt url
df['url'] = df['url'].apply(lambda x: fernet.decrypt(x).decode())
df[['member_id', 'url']].head(5)

,member_id,url
34866,787218,gAAAAABl2JerrOFlfRVgSRk81QRL9ixHCJGa9BS55v70D6...
2735,764324,gAAAAABl2Jert_5dmPO0vNAj8YLTKVk_Z5g9Ay0vPT_OD6...
40836,835461,gAAAAABl2JerWkwkApgohfh33n_kf7vP7USQG1_eHbkKPF...
21990,1080466,gAAAAABl2Jer89uFh6aoBAUlddX_cI4933Kud6JnmDCUSu...
4759,67800109,gAAAAABl2JerBYZljvI7N7wwq4TQUKW9M2T5SQi2-x0jXA...


If we have to permanently anonymize the data we can replace the url with a hash value.

In [ ]:
# encrypt url
df['url'] = df['url'].apply(lambda x: hash(x))
df[['member_id', 'url']].head(5)